In [2]:
import coral as cr
import aquariumapi

### CAN1

I selected CAN1 for my gene of interest. I know it's both selectable and counter-selectable, which could make for interesting dosage-dependent growth effects.

Let's download the sequences we want using Coral (we could use yeastmine, but Coral actually has a way to do that as well). As a reminder, we care about the promoter region, especially near the transcriptional start site.

In [5]:
# Get the coding sequence from the yeast genome database
can1 = cr.database.fetch_yeast_locus_sequence('CAN1')

In [8]:
# Get the promoter from the yeast promoter atlas (YPA)
pcan1 = cr.database.get_yeast_promoter_ypa('CAN1')

In [30]:
# Because GG is so common, I'm going to limit the target size to 100 bp
template = pcan1[-60:] + can1[:60]

In [31]:
# Let's find all of the NGG sites (spCas9 PAM)
ngg = template.locate('GG')
print ngg

[[51, 92, 116], [4, 36, 91]]


In [32]:
# I like the ones that are closest to the start of the gene sequence,
# i.e. those near 60 bp (half the length)
loc_top = ngg[0][0]
loc_bottom = ngg[1][1]

In [33]:
# Let's 20 bp upstream of each NGG as ssDNA
target1 = template.top[loc_top - 20:loc_top]
target2 = template.bottom[loc_bottom - 20:loc_bottom]

In [34]:
# Now we have our targets. Our gRNA should actually be the reverse complement
# of this
grna1 = target1.reverse_complement()
grna2 = target2.reverse_complement()

In [35]:
# Let's view our sequences
print grna1
print grna2

TTTTTTTTTTTTTGTTTTTA
GACATAGAGGAGAAGCATAT


In [36]:
# Okay, grna1 sucks. Let's grab a different one
loc_top2 = ngg[0][2]
grna3 = template.top[loc_top2 - 20:loc_top2].reverse_complement()
print grna3

GGCTCATTGTACATATGCTT


### Cloning design

We need the backbone sequence in order to do this. I downloaded it from benchling and found the NNN site.

In [40]:
backbone_template = cr.seqio.read_dna('./pmod8-pgalz4-urgr.gb')
backbone_template.display()

In [43]:
dummy_target_f = backbone_template.select_features('Target Sequence')[0]
backbone = backbone_template.excise(dummy_target_f)
backbone.display()

In [49]:
# Primer design - like the last example, the backbone already exists.
# Unlike the last example, we're going to do a templateless PCR to
# assemble the gRNA target. Therefore, the 'anneal' region is the
# target sequence and the 'overhang' will match the backbone

# First, let's check the Tm of the whole gRNA to see what our
# approach has to be
print cr.DNA(str(grna2)).tm()
print cr.DNA(str(grna3)).tm()

52.9703061723
57.2689058873


In [50]:
# These are too low (in the 50s) How can we increase the length?

It's actually non-obvious, given what we've learned. Using `design.primer` sounds like a good idea, but you'll quickly run into an issue: overshooting the Tm. This is because every time you increase a base on the forward primer, you're adding homology to the revese primer - so the Tm is really the 'union' of the two sequences.

There's multiple ways we could make this happen. Luckily, Coral already has a templateless PCR design function from days long past. I hope it still works!

In [54]:
# First, design Gibson homology using design.primer
# This is for the 'top' strand in both cases
homology1 = cr.design.primer(backbone.flip()).primer().reverse_complement()
homology2 = cr.design.primer(backbone).primer()

In [65]:
# Now use the oligo assembly designer
to_assemble2 = homology1 + cr.DNA(str(grna2)) + homology2
assembly = cr.design.OligoAssembly(to_assemble2, tm=65, length_range=(20, 60))
out = assembly.design_assembly()

In [71]:
fwd = cr.Primer(out['oligos'][0], tm=out['oligos'][0].tm())
rev = cr.Primer(out['oligos'][1], tm=out['oligos'][1].tm())

In [72]:
cr.reaction.pcr(fwd.primer().to_ds(), fwd, rev)

GCCCGTCCCATCTCCCGACATAGAGGAGAAGCATATGTTTTAGAGCTAGAAATAGCAAGTTAAAATAAGG
CGGGCAGGGTAGAGGGCTGTATCTCCTCTTCGTATACAAAATCTCGATCTTTATCGTTCAATTTTATTCC